In [106]:
import pandas as pd
import numpy as np

In [107]:
data = pd.read_csv("data/SalaryData.csv")

# Cleaning 

In [108]:
# drops cols we dont need 
cols_to_drop = ['jobTitle', 'Other', 'ObjectId']
data = data.drop(columns=cols_to_drop)
# calculates total expected salary 
data['Salary_Total'] = data['Annual_Rate'] + data['Incentive_Allowance']
# Filling missing data with 0 
data = data.fillna(0)
# merging the departments together 
data['Department'] = data['Department'].replace('Louisville Metro Police', 'Louisville Metro Police Department')
# calculates total weekly hrs worked 
data['Hr_Rate'] = data['Regular_Rate'] / 2080
data['Ot_Rate'] = data['Hr_Rate'] * 1.5
data['Hr_Worked'] = data['Overtime_Rate'] / data['Ot_Rate'] / 52 + 40
data.replace([np.inf, -np.inf], 40, inplace=True)
# data = data[data['CalYear']==2023]

In [ ]:
data.head(2)

,CalYear,Employee_Name,Department,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,YTD_Total,Salary_Total,Hr_Rate,Ot_Rate,Hr_Worked
0,2023,"Summers, William E",OMB Finance,5200.0,1600.00,0.0,0.0,1600.00,5200.0,0.769231,1.153846,40.0
1,2023,"Martin, David",Louisville Free Public Library,30160.0,29857.64,0.0,800.0,30657.64,30960.0,14.354635,21.531952,40.0


In [ ]:
department = data.groupby(['CalYear', 'Department'])[['YTD_Total', 'Salary_Total']].sum().reset_index()

In [111]:
department.head()

,CalYear,Department,YTD_Total,Salary_Total
0,2019,Air Pollution Control,3161000.01,3357059.36
1,2019,Belle of Louisville,1288272.26,2255142.24
2,2019,Commonwealth Attorney,1076553.31,1254064.50
3,2019,Coroner,883494.80,1006419.20
4,2019,County Attorney,5253504.81,6583707.28


# Lmpd general view

In [112]:
lmpd = department[department['Department'] == 'Louisville Metro Police Department'].copy()
lmpd = lmpd[lmpd['CalYear'] != 2024].copy()
lmpd

,CalYear,Department,YTD_Total,Salary_Total
19,2019,Louisville Metro Police Department,1.022160e+08,9.862518e+07
56,2020,Louisville Metro Police Department,1.061527e+08,1.040606e+08
93,2021,Louisville Metro Police Department,1.033841e+08,1.020937e+08
131,2022,Louisville Metro Police Department,1.224085e+08,1.051902e+08
176,2023,Louisville Metro Police Department,1.223053e+08,1.181829e+08


In [114]:
lmpd = pd.read_csv('data/test.csv')

In [115]:

lmpd

,Unnamed: 0,CalYear,Department,YTD_Total,Salary_Total
0,19,2019,Louisville Metro Police Department,1.022160e+08,9.862518e+07
1,56,2020,Louisville Metro Police Department,1.061527e+08,1.040606e+08
2,93,2021,Louisville Metro Police Department,1.033841e+08,1.020937e+08
3,131,2022,Louisville Metro Police Department,1.224085e+08,1.051902e+08
4,176,2023,Louisville Metro Police Department,1.223053e+08,1.181829e+08


In [116]:
lmpd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    5 non-null      int64  
 1   CalYear       5 non-null      int64  
 2   Department    5 non-null      object 
 3   YTD_Total     5 non-null      float64
 4   Salary_Total  5 non-null      float64
dtypes: float64(2), int64(2), object(1)
memory usage: 332.0+ bytes


In [ ]:
lmpd['Discrepancy'] = lmpd['YTD_Total'] - lmpd['Salary_Total']
lmpd['Discrepancy_Ratio'] = lmpd['Discrepancy'] / lmpd['Salary_Total']

lmpd

,Unnamed: 0,CalYear,Department,YTD_Total,Salary_Total,Discrepancy,Discrepancy_Ratio
0,19,2019,Louisville Metro Police Department,1.022160e+08,9.862518e+07,3590832.44,0.036409
1,56,2020,Louisville Metro Police Department,1.061527e+08,1.040606e+08,2092070.71,0.020104
2,93,2021,Louisville Metro Police Department,1.033841e+08,1.020937e+08,1290365.61,0.012639
3,131,2022,Louisville Metro Police Department,1.224085e+08,1.051902e+08,17218319.86,0.163687
4,176,2023,Louisville Metro Police Department,1.223053e+08,1.181829e+08,4122435.66,0.034882


In business, a discrepancy ratio between expected salary spend (Salary_Total) and actual salary spend (YTD_Total) typically falls within 0 to 5% as an acceptable range. A discrepancy within this range usually indicates minor variations in salary spending, such as unexpected overtime, temporary staffing, or adjustments to salary levels, which are often manageable within the overall budget. A discrepancy ratio of more than 10% would generally raise concerns, signaling potential overspending on salaries, and would require a deeper investigation into the causes, such as unplanned hiring, excessive overtime, or compensation adjustments.

For the Louisville Metro Police Department, the discrepancy ratios from 2019 to 2023 show a range from 0.01 to 0.16. The ratios in 2019, 2020, 2021, and 2023 are all within a 0.01 to 0.04 range, which suggests a relatively small and manageable variance in salary spending. However, the discrepancy ratio for 2022 is significantly higher at 0.16, indicating a 16% overspend on salaries compared to the planned budget for that year. This could point to specific factors in 2022 that led to a sharp increase in salary expenses, such as a one-time hiring initiative, increased overtime costs, or other unforeseen budgetary pressures.

# LMPD indv level

In [120]:
indv = pd.read_csv('data/SalaryData.csv')

In [121]:
cols_to_drop = ['Regular_Rate', 'Overtime_Rate', 'Other', 'ObjectId']
indv = indv.drop(columns=cols_to_drop)
indv['Salary_Total'] = indv['Annual_Rate'] + indv['Incentive_Allowance']
final_drop = ['Annual_Rate', 'Incentive_Allowance']
indv = indv.drop(columns=final_drop)
indv = indv.fillna(0)
indv['Department'] = indv['Department'].replace('Louisville Metro Police', 'Louisville Metro Police Department')
indv = indv[indv['CalYear'] != 2024].copy()

In [123]:
indv = indv[indv['Department'] == 'Louisville Metro Police Department'].copy()
# indv.head(2)

In [124]:
indv['Discrepancy'] = indv['YTD_Total'] - indv['Salary_Total']
indv['Discrepancy_Ratio'] = indv['Discrepancy'] / indv['Salary_Total']


In [125]:
indv.head(2)

,CalYear,Employee_Name,Department,jobTitle,YTD_Total,Salary_Total,Discrepancy,Discrepancy_Ratio
2,2023,"Bratcher, Elaine",Louisville Metro Police Department,Clerk II-Police,41457.53,41457.52,0.01,2.412108e-07
3,2023,"Jackson, Nila Marie",Louisville Metro Police Department,Traffic Guard I,19823.87,16912.00,2911.87,1.721777e-01


Top 10 Employees with the Highest Discrepancies

In [137]:
def top_10_discrepancy(indv, enter_year):
    # Filter the DataFrame for the given year
    indv_year = indv[indv['CalYear'] == enter_year]
    
    # Sort by 'Discrepancy' in descending order to get the top 10 employees
    top_10_employees = indv_year.sort_values(by='Discrepancy_Ratio', ascending=False).head(10)
    
    # Return the top 10 employees with their discrepancies
    return top_10_employees[['CalYear', 'Employee_Name', 'jobTitle', 'Discrepancy', 'Discrepancy_Ratio', 'YTD_Total', 'Salary_Total']]

In [141]:
# Example usage:
top_emp_dis_2019 = top_10_discrepancy(indv, 2021)
top_emp_dis_2019

,CalYear,Employee_Name,jobTitle,Discrepancy,Discrepancy_Ratio,YTD_Total,Salary_Total
10694,2021,"Cottongim, Zachary",Police Officer,137606.36,2.020447,205713.24,68106.88
8048,2021,"Hodge, Jeff",Police Officer,79945.71,1.031684,157436.19,77490.48
7153,2021,"Chambers, Amy",Police Sergeant,89193.96,0.946147,183464.68,94270.72
8272,2021,"Fey, Ronald",Police Sergeant,78971.71,0.910470,165708.99,86737.28
8961,2021,"Ruoff, Elizabeth",Police Officer,65257.51,0.893541,138289.99,73032.48
8758,2021,"Keeling, Joseff",Police Sergeant,71738.28,0.866759,154504.36,82766.08
11548,2021,"Dykes, Jan",Police Officer,54448.67,0.824229,120508.83,66060.16
9646,2021,"O'Daniel, Timothy",Police Officer,57561.05,0.806436,128938.17,71377.12
10943,2021,"Isham, Vernon",Storage Equipment Operator5day,35648.81,0.770033,81944.01,46295.20
9561,2021,"Flannery, Aaron",Police Officer,50151.04,0.725639,119264.00,69112.96


In [128]:
top_emp_dis_2020 = top_10_discrepancy(indv, 2020)
top_emp_dis_2020

,CalYear,Employee_Name,jobTitle,Discrepancy,YTD_Total,Salary_Total
22390,2020,"Keller, Matthew",Police Sergeant,69394.43,162998.75,93604.32
22193,2020,"Smith, Jeremy",Police Sergeant,62327.48,155383.88,93056.40
21894,2020,"Fey, Ronald",Police Sergeant,61428.87,151054.15,89625.28
22207,2020,"Szydlowski, Victor",Police Sergeant,59551.93,151002.81,91450.88
21580,2020,"Watkins, Kelvin",Police Officer,57862.27,137577.79,79715.52
21300,2020,"Miller, Russell",Police Lieutenant,54094.56,155163.36,101068.80
21296,2020,"Hume, Jill",Police Lieutenant,53387.49,154541.49,101154.00
21266,2020,"Burbrink, Donald",Police Lieutenant,51273.52,152204.80,100931.28
21410,2020,"Rivera, Arnold",Police Lieutenant,49459.40,150423.80,100964.40
22908,2020,"Claxon, Wesley",Police Officer,48626.56,126546.00,77919.44


In [129]:
top_emp_dis_2022 = top_10_discrepancy(indv, 2022)
top_emp_dis_2022

,CalYear,Employee_Name,jobTitle,Discrepancy,YTD_Total,Salary_Total
28061,2022,"Fey, Ronald W",Police Sergeant,99793.00,199973.34,100180.34
30909,2022,"Dahlgren, Patrick Barthold",Police Officer,92495.11,171864.72,79369.61
31609,2022,"Crawford, Montrell",Police Officer,91343.61,169561.49,78217.88
28101,2022,"Keller, Matthew J",Police Lieutenant,86355.73,212051.00,125695.27
30288,2022,"Koestel, Matthew Robert",Police Officer,84322.63,165746.73,81424.10
29285,2022,"Flannery, Aaron Jerome",Police Officer,84290.25,164265.53,79975.28
29549,2022,"Mattiche, Youssef",Police Officer,80721.37,163813.87,83092.50
28102,2022,"Neal, Paul Eugene",Police Sergeant,79044.77,184108.74,105063.97
31859,2022,"Capito, Jacob Paul",Police Officer,78714.69,155655.01,76940.32
28941,2022,"Sears, Shelby Edward Vanwinkle",Police Sergeant,74790.00,171524.56,96734.56


Job Title with the Largest Individual Discrepancy by Year:

In [130]:
# Group by CalYear and jobTitle to find the row with the largest discrepancy per year
largest_discrepancy_by_year = indv.loc[indv.groupby('CalYear')['Discrepancy'].idxmax()]

# Display the job title and the corresponding discrepancy
largest_discrepancy_by_year[['CalYear', 'jobTitle', 'Discrepancy']]

,CalYear,jobTitle,Discrepancy
14865,2019,Police Sergeant,48026.85
22390,2020,Police Sergeant,69394.43
10694,2021,Police Officer,137606.36
28061,2022,Police Sergeant,99793.00
1806,2023,Police Sergeant,116884.19


top 10 job titles by avg discrepancy by year

In [131]:
def top_10_job_titles_by_avg_discrepancy(indv, enter_year):
    # Filter the DataFrame for the given year
    indv_year = indv[indv['CalYear'] == enter_year]
    
    # Group by 'jobTitle' and calculate the average 'Discrepancy' for each job title
    avg_discrepancy_by_job = indv_year.groupby('jobTitle')['Discrepancy'].mean()
    
    # Sort by the average 'Discrepancy' in descending order to get the top 10 job titles
    top_10_job_titles = avg_discrepancy_by_job.sort_values(ascending=False).head(10)
    
    # Return the relevant columns (jobTitle and average Discrepancy)
    return top_10_job_titles.reset_index()[['jobTitle', 'Discrepancy']]

In [132]:
# Example usage:
top_10_job_titles_by_avg_discrepancy_2023 = top_10_job_titles_by_avg_discrepancy(indv, 2023)
top_10_job_titles_by_avg_discrepancy_2023

,jobTitle,Discrepancy
0,Tow-In Equipment Operator 6day,53743.320000
1,Storage Equipment Operator 5 Day,24019.662000
2,Information Process Tech II,19541.015000
3,Traffic Guard Supervisor,16161.520000
4,Information Systems Supervisor,13577.490000
5,Group Violence Victimology Specialist,12925.525000
6,Non-Sworn Rotorcraft Pilot,12502.400000
7,Technology Program Manager,12205.090000
8,Police Lieutenant,11157.732969
9,Police Sergeant,10827.717278


Average Discrepancy per Job Title:

In [133]:
def avg_discrepancy_by_job(indv):
    # Group by 'jobTitle' and calculate the average discrepancy (YTD_Total - Salary_Total)
    avg_discrepancy_per_job = indv.groupby('jobTitle')['Discrepancy'].mean()
    
    # Sort the result by average discrepancy in descending order
    avg_discrepancy_per_job_sorted = avg_discrepancy_per_job.sort_values(ascending=False)
    
    # Return the sorted average discrepancies
    return avg_discrepancy_per_job_sorted

In [134]:
# Example usage:
avg_discrepancy = avg_discrepancy_by_job(indv)
avg_discrepancy

jobTitle
Special Assistant- PT               32913.850000
Special Assistant - PT              32479.040000
Storage Equipment Operator 5 Day    24131.235000
Information Process Tech II         17177.815000
Tow-In Equipment Operator 6day      16195.246000
                                        ...     
Firearms & Toolmark Examiner       -34152.880000
Training Academy Instructor        -35417.588571
Curriculum Development Coord       -39393.660000
Project Manager II                 -49539.000000
Police Chief                       -69311.240000
Name: Discrepancy, Length: 113, dtype: float64